In [43]:
from brfinance import CVMAsyncBackend
import pandas as pd
from datetime import datetime, date

cvm_httpclient = CVMAsyncBackend()

# Dict de códigos CVM para todas as empresas
cvm_codes = cvm_httpclient.get_cvm_codes()

# Dict de todas as categorias de busca disponíveis (Fato relevante, DFP, ITR, etc.)
categories = cvm_httpclient.get_consulta_externa_cvm_categories()

# Realizando busca por Empresa
ano = int(input("Qual o ano inicial para a pesquisa? "))
start_date = date(ano, 12, 31)
end_dt = date.today()
cvm_codes_list = ['21067'] # Moura_Dubeux
category = ["EST_4"] # Códigos de categoria para DFP, ITR e fatos relevantes #, "EST_3", "IPE_4_-1_-1"
last_ref_date = False # Se "True" retorna apenas o último report no intervalo de datas

# Criar um dataframe vazio para armazenar os dados
df_concat = pd.DataFrame()

# Busca
search_result = cvm_httpclient.get_consulta_externa_cvm_results(
    cod_cvm=cvm_codes_list,
    start_date=start_date,
    end_date=end_dt,
    last_ref_date=last_ref_date,
    category=category
    )

# Filtrar dataframe de busca para DFP e Status Ativo
search_result = search_result[
    (search_result['categoria']=="DFP - Demonstrações Financeiras Padronizadas")]
search_result = search_result[search_result['status']=="Ativo"]
search_result = search_result[pd.to_numeric(search_result['numero_seq_documento'], errors='coerce').notnull()]

reports_list = [
    'Balanço Patrimonial Ativo',
    'Balanço Patrimonial Passivo',
    'Demonstração do Resultado',
    'Demonstração do Fluxo de Caixa'] # Se None retorna todos os demonstrativos disponíveis.

# Obter demonstrativos
for index, row in search_result.iterrows():
    empresa = f"{row['cod_cvm']} - {cvm_codes[row['cod_cvm']]}"

    reports = cvm_httpclient.get_report(row["numero_seq_documento"], row["codigo_tipo_instituicao"], reports_list=reports_list)

    for report in reports:
        print(report)
        reports[report]["cod_cvm"] = row["cod_cvm"]
        
        # Concatenar os dados de todos os anos em um único dataframe
        df_concat = pd.concat([df_concat, reports[report]], axis=0)


        # Salvar o dataframe no arquivo excel, repetindo as linhas das DFPs conforme os anos do filtro
df_concat.to_excel('Dados_CVM.xlsx', index=False)

# Criar pivot table
pivot_table = pd.pivot_table(df_concat, index=["Conta", "Descrição"], values=["Valor"], aggfunc=lambda x: "\n".join(str(i) for i in x))

# Salvar pivot table em um arquivo Excel
pivot_table.to_excel("pivot_table.xlsx")

# Carregar o arquivo pivot_table.xlsx em um DataFrame pandas
df_pivot = pd.read_excel('pivot_table.xlsx')

# Obter o número máximo de valores em uma única célula "Valor" em todo o DataFrame
max_values = int(df_pivot['Valor'].str.count('\n').max()) + 1
value_cols = [f'{ano+i}' for i in reversed(range(max_values))]
ano = ano-max_values-1 # move this line after value_cols is created
df_pivot[value_cols] = df_pivot['Valor'].str.split('\n', expand=True)

# Separar os valores da coluna "Valor" em colunas separadas
df_pivot[value_cols] = df_pivot['Valor'].str.split('\n', expand=True)

# Remover a coluna "Valor" original
df_pivot = df_pivot.drop(columns=['Valor'])

# Ordenar colunas em ordem crescente, mantendo as duas primeiras colunas
all_cols = list(df_pivot.columns)
ordered_cols = all_cols[:2] + sorted(all_cols[2:])

# Reagrupamento das colunas no DataFrame
df_pivot = df_pivot.reindex(columns=ordered_cols)

# Salvar o DataFrame atualizado em um arquivo Excel
df_pivot.to_excel('pivot_table_updated.xlsx', index=False)





Qual o ano inicial para a pesquisa? 2010


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  response_df = response_df.append(search_results_df)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/Frameworks/Python.framework/Versions/

Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/F

Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa


...

Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/F

Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/brfinance/responses.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis([*df.columns[:-1], 'Valor'], axis=1, inplace=True)
/Library/F

Balanço Patrimonial Ativo
Balanço Patrimonial Passivo
Demonstração do Resultado
Demonstração do Fluxo de Caixa


In [2]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
